In [ ]:
# ✅ שלב 0: התקנת YOLOX (ללא onnx-simplifier)
!git clone https://github.com/Megvii-BaseDetection/YOLOX
%cd YOLOX
!sed -i '/onnx-simplifier/d' requirements.txt
!pip install -r requirements.txt
!pip install -v -e .


Cloning into 'YOLOX'...
remote: Enumerating objects: 1928, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1928 (delta 2), reused 0 (delta 0), pack-reused 1916 (from 2)
Receiving objects: 100% (1928/1928), 7.55 MiB | 12.25 MiB/s, done.
Resolving deltas: 100% (1148/1148), done.
/content/YOLOX
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
  

In [ ]:
%%bash
# להריץ בתיקיית /content/YOLOX
cd /content/YOLOX
set -e

echo "🔧  Patching trainer.py – disable L1 once and for all …"

# 1️⃣  מבטלים את ההדפסה + התנאי (עם או בלי רווח אחרי החצים)
sed -i -E \
  -e 's/self\.logger\.info\("---> ?Add additional L1 loss now!"\)/# L1 disabled – no log/' \
  -e 's/if +self\.exp\.add_l1_loss/if False  # patched: never add L1/' \
  yolox/core/trainer.py

# 2️⃣  מחליפים כל self.exp.add_l1_loss → False (שלא יופעל בשום מקום)
sed -i 's/self\.exp\.add_l1_loss/False/g' yolox/core/trainer.py

echo "✅  trainer.py patched – L1 loss מבוטל לחלוטין"


Patching trainer.py ...
Done!


In [ ]:
import inspect, textwrap, yolox.core.trainer as T
print(textwrap.dedent("\n".join(
    l for l in inspect.getsource(T.Trainer).splitlines()
    if 'add_l1_loss' in l or 'logger.info("--->' in l
)))


logger.info("---> start train epoch{}".format(self.epoch + 1))
    logger.info("--->No mosaic aug now!")
    logger.info("--->Add additional L1 loss now!")


In [ ]:
# ✅ שלב 1: העלאת קובץ ZIP וחילוץ
from google.colab import files
import zipfile, os

uploaded = files.upload()
zip_filename = next(iter(uploaded))
extract_path = "datasets/drone_ir"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# שינוי שם תיקיה עם רווחים אם צריך
old_name = os.path.join(extract_path, "Drone Detection.v1i.yolov5pytorch")
new_name = os.path.join(extract_path, "drone_dataset")
if os.path.exists(old_name):
    os.rename(old_name, new_name)

print(f"✅ הקובץ {zip_filename} חולץ לתוך: {new_name}")


Saving Drone Detection.v1i.yolov5pytorch (2).zip to Drone Detection.v1i.yolov5pytorch (2).zip
✅ הקובץ Drone Detection.v1i.yolov5pytorch (2).zip חולץ לתוך: datasets/drone_ir/drone_dataset


In [ ]:
# ✅ שלב 2: יצירת train.txt, val.txt, classes.txt
import glob

base_path = "datasets/drone_ir/drone_dataset"
train_images_dir = os.path.join(base_path, "train", "images")
val_images_dir = os.path.join(base_path, "valid", "images")

train_imgs = glob.glob(os.path.join(train_images_dir, "*.jpg"))
val_imgs = glob.glob(os.path.join(val_images_dir, "*.jpg"))

with open(os.path.join(base_path, "train.txt"), "w") as f:
    f.write("\n".join(train_imgs))

with open(os.path.join(base_path, "valid.txt"), "w") as f:
    f.write("\n".join(val_imgs))

with open(os.path.join(base_path, "classes.txt"), "w") as f:
    f.write("droneIR")

print("✅ נוצרו הקבצים train.txt, valid.txt, classes.txt")


✅ נוצרו הקבצים train.txt, valid.txt, classes.txt


In [ ]:
# ✅ Exp חדש – אין Evaluator ואין L1‑loss
import os, textwrap
from pathlib import Path

EXP_PATH = Path("exps/example/custom")
EXP_PATH.mkdir(parents=True, exist_ok=True)

exp_code = textwrap.dedent("""
    import os
    from yolox.exp import Exp as BaseExp

    class Exp(BaseExp):
        def __init__(self):
            super().__init__()
            # --- בסיס ---
            self.num_classes  = 1
            self.depth        = 0.33
            self.width        = 0.5
            self.input_size   = (640, 640)
            self.test_size    = (640, 640)
            self.exp_name     = "yolox_drone"
            self.max_epoch    = 10

            # --- בלי evaluator ---
            self.eval_interval = -1
            self.no_eval       = True

            # --- נתיבי Dataset ---
            self.data_dir  = "datasets/drone_ir/drone_dataset"
            self.train_ann = "train"
            self.val_ann   = "valid"
            self.classes   = ["droneIR"]

            # --- אוגמנטציה מינימלית ---
            self.enable_mixup = False
            self.mosaic_prob  = 0.0
            self.mixup_prob   = 0.0
            self.hsv_prob     = 1.0
            self.flip_prob    = 0.5
            self.no_aug       = True

            # --- Loss / LR ---
            self.add_l1_loss   = False      # לא מפעילים L1
            self.warmup_epochs = 0

            # --- NMS / confidence ---
            self.test_conf = 0.01
            self.nmsthre   = 0.65

        # ---------- model ----------
        def get_model(self):
            # משתמשים במימוש ברירת‑המחדל של YOLOX
            model = super().get_model()
            # מכבים את L1‑loss ממש בתוך הראש
            if hasattr(model, "head") and hasattr(model.head, "use_l1"):
                model.head.use_l1 = False
            return model

        # ---------- datasets ----------
        def get_dataset(self, cache=False, cache_type="ram"):
            from yolox.data.data_augment import TrainTransform
            from yolox.data.datasets.yolov5dataset import YOLOv5Dataset
            return YOLOv5Dataset(
                data_dir=self.data_dir,
                name=self.train_ann,
                img_size=self.input_size,
                preproc=TrainTransform(
                    max_labels=50,
                    flip_prob=self.flip_prob,
                    hsv_prob=self.hsv_prob,
                ),
                cache=cache,
            )

        def get_eval_dataset(self, **kwargs):
            from yolox.data.data_augment import ValTransform
            from yolox.data.datasets.yolov5dataset import YOLOv5Dataset
            return YOLOv5Dataset(
                data_dir=self.data_dir,
                name=self.val_ann,
                img_size=self.test_size,
                preproc=ValTransform(),
            )
""")

with open(EXP_PATH / "yolox_drone.py", "w") as f:
    f.write(exp_code)

print("✅  exps/example/custom/yolox_drone.py נכתב – head.use_l1=False, אין Evaluator")


✅  exps/example/custom/yolox_drone.py נכתב – head.use_l1=False, אין Evaluator


In [ ]:
# ✅ יצירת YOLOv5Dataset עם הפונקציה pull_item + evaluate_detections שלב 4
!mkdir -p yolox/data/datasets

dataset_code = '''
import os
import cv2
import numpy as np
from torch.utils.data import Dataset

class YOLOv5Dataset(Dataset):
    def __init__(self, data_dir, name, img_size, preproc=None, cache=False):
        cls_path = os.path.join(data_dir, "classes.txt")
        with open(cls_path, "r") as f:
            self.class_names = [c.strip() for c in f.readlines()]
        self.num_classes = len(self.class_names)
        self.img_size = img_size
        self.preproc = preproc
        self.input_dim = img_size

        list_path = os.path.join(data_dir, f"{name}.txt")
        with open(list_path, "r") as f:
            self.img_files = [line.strip() for line in f.readlines()]

        self.img_files = [
            path if os.path.isabs(path) or path.startswith(data_dir) else os.path.join(data_dir, path)
            for path in self.img_files
        ]

        self.label_files = []
        for img_path in self.img_files:
            base_path = os.path.dirname(img_path).replace("images", "labels")
            file_name = os.path.basename(img_path).replace(".jpg", ".txt").replace(".png", ".txt")
            label_path = os.path.join(base_path, file_name)
            self.label_files.append(label_path)

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, index):
        if isinstance(index, tuple):
            index = index[0]
        img, label, img_info, img_id = self.pull_item(index)
        if self.preproc:
            img, label = self.preproc(img, label, self.input_dim)
        return img, label, img_info, img_id

    def pull_item(self, index):
        img_path = self.img_files[index]
        label_path = self.label_files[index]

        img = cv2.imread(img_path)
        assert img is not None, f"Image not found: {img_path}"
        height, width = img.shape[:2]

        labels = []
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        cls, x, y, w, h = map(float, parts)
                        labels.append([cls, x, y, w, h])
        labels = np.array(labels, dtype=np.float32).reshape(-1, 5) if labels else np.zeros((0, 5), dtype=np.float32)

        img_info = (height, width)
        img_id = index

        return img, labels, img_info, img_id

    def evaluate_detections(self, all_boxes, save_dir):
        print("⚠️ Warning: evaluation skipped (dummy implementation).")
        return 0.0, 0.0  # mAP50, mAP70
'''

with open("yolox/data/datasets/yolov5dataset.py", "w") as f:
    f.write(dataset_code)

print("✅ YOLOv5Dataset נוצר בהצלחה עם evaluate_detections מדומה")


✅ YOLOv5Dataset נוצר בהצלחה עם evaluate_detections מדומה


In [ ]:
import os

# 📁 נתיב לתיקיית הדאטהסט שלך
base_dir = "datasets/drone_ir/drone_dataset"

# 🧠 נתיבים לצורך בדיקה
train_labels = os.path.join(base_dir, "train", "labels")
valid_labels = os.path.join(base_dir, "valid", "labels")

print(f"🔍 Checking label folders in: {base_dir}")

# ✅ בדיקה שהתקיות קיימות
if not os.path.isdir(train_labels):
    print(f"❌ Label directory does not exist: {train_labels}")
    print("🛑 Training aborted due to invalid label files.")
elif not os.path.isdir(valid_labels):
    print(f"❌ Label directory does not exist: {valid_labels}")
    print("🛑 Training aborted due to invalid label files.")
else:
    print("✅ Label folders found!")

    # 🚀 אם הכול תקין – נריץ את האימון
    !python tools/train.py -f exps/example/custom/yolox_drone.py -d 1 -b 8 --fp16 -o


🔍 Checking label folders in: datasets/drone_ir/drone_dataset
✅ Label folders found!
2025-05-08 14:15:37.950693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746713737.972353    1840 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746713737.979422    1840 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/content/YOLOX/yolox/core/trainer.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=args.fp16)
2025-05-08 14:15:41 | INFO     | yolox.core.trainer:132 - args: Namespace(experiment_name='yolox_drone', name=None, dist_bac